In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pathlib import Path
import time

filename = "NammaMetro_Ridership_Dataset.csv"

In [3]:
# Selenium options required to create a 'headless' browser
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

driver = webdriver.Chrome(options=options)

In [4]:
# Load ridership page from BMRCL website
driver.get("https://english.bmrc.co.in/ridership/")

# Click on Kannada toggle button to load English results
time.sleep(10)  # Allow extra time for JavaScript to load translated data
result = driver.find_element(By.CLASS_NAME, "link.top-navcustom-text").click()

In [5]:
# Results are published with a lag of about one day. 
# So get the date on the page rather than date.today()
record_date = driver.find_element(By.TAG_NAME, "h3").text
record_date

'Passenger Flow as on 12-08-2025'

In [6]:
# Initialize dict to store ridership data
day_record = {}
day_record['Record Date'] = [record_date.split()[-1]]  # Extracting date part
day_record

{'Record Date': ['12-08-2025']}

In [7]:
# Parse html for remaining data points and store in pandas dataframe
data_points = driver.find_elements(By.CLASS_NAME, "features-card.achivement-area.bg-color")

for l1 in data_points:
    for l2 in l1.text.split('\n'):
        data = l2.split(': ')
        day_record[data[0]] = [int(data[1])]

driver.quit()

day_record = pd.DataFrame(day_record)
day_record.rename(columns={'Tokens':'Total Tokens'}, inplace=True)
day_record

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Total Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
0,12-08-2025,525613,524958,259,56,340,263412,34320,576,208766,65190,105273,38303


In [8]:
# Store data in csv file - create file if necessary
filePath = Path(filename)
if filePath.exists() and filePath.is_file():
    day_record.to_csv(filename, mode='a', header=False, index=False, lineterminator='\n')
    print('Appended '+filename)
else:
    day_record.to_csv(filename, mode='w', header=True, index=False, lineterminator='\n')
    print('Created '+filename)

Appended NammaMetro_Ridership_Dataset.csv


In [9]:
# Optimize dataset by removing duplicates and rewrite to file
df = pd.read_csv(filename).drop_duplicates(subset=['Record Date'], keep='last', ignore_index=True)
df[df.loc[:, 'Total Smart Cards':].columns] = df[df.loc[:, 'Total Smart Cards':].columns].astype('Int64')
df.to_csv(filename, mode='w', header=True, index=False, lineterminator='\n')
df.tail()

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Total Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
219,02-08-2025,300912,300440,320,16,136,248287,17623,414,217745,59387,120400,37958
220,04-08-2025,443237,442573,116,9,539,218087,26260,107,180895,52657,96641,31597
221,06-08-2025,475439,474681,169,44,545,213525,29037,185,179785,51217,96343,32225
222,11-08-2025,503837,503041,436,49,311,303165,32198,450,208382,71042,96001,41339
223,12-08-2025,525613,524958,259,56,340,263412,34320,576,208766,65190,105273,38303
